In [2]:
import env, acquire, wrangle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn.preprocessing
from sklearn.model_selection import train_test_split

# Exercises II 

>Let's set up an example scenario as perspective for our regression exercises using the Telco dataset.
As a customer analyst for Telco, you want to know who has spent the most money with the company over their lifetime. You have monthly charges and tenure, so you think you will be able to use those two attributes as features to estimate total charges. You need to do this within an average of 5.00 dollars per customer In these exercises, you will complete the first step toward the above goal: acquire and prepare the necessary Telco data from the telco_churn database in the Codeup database server.

#### 1. Acquire customer_id, monthly_charges, tenure, and total_charges from the telco_churn database for all customers with a 2-year contract

In [16]:
telco_df = acquire.get_telco_db('telco_churn') # import telco df 

In [17]:
telco_df # look at it to get a picture of the data

,payment_type_id,contract_type_id,internet_service_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,internet_service_type,contract_type,payment_type
0,2,1,3,0030-FNXPP,Female,0,No,No,3,Yes,...,No internet service,No internet service,No internet service,No,19.85,57.2,No,None,Month-to-month,Mailed check
1,2,1,3,0031-PVLZI,Female,0,Yes,Yes,4,Yes,...,No internet service,No internet service,No internet service,No,20.35,76.35,Yes,None,Month-to-month,Mailed check
2,1,1,3,0098-BOWSO,Male,0,No,No,27,Yes,...,No internet service,No internet service,No internet service,Yes,19.40,529.8,No,None,Month-to-month,Electronic check
3,1,1,3,0107-WESLM,Male,0,No,No,1,Yes,...,No internet service,No internet service,No internet service,Yes,19.85,19.85,Yes,None,Month-to-month,Electronic check
4,3,1,3,0114-RSRRW,Female,0,Yes,No,10,Yes,...,No internet service,No internet service,No internet service,Yes,19.95,187.75,No,None,Month-to-month,Bank transfer (automatic)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,4,3,1,9919-FZDED,Male,1,Yes,No,72,Yes,...,No,Yes,Yes,Yes,84.10,5981.65,No,DSL,Two year,Credit card (automatic)
7039,3,3,1,9926-PJHDQ,Female,0,Yes,Yes,72,Yes,...,No,Yes,Yes,Yes,76.80,5468.45,No,DSL,Two year,Bank transfer (automatic)
7040,2,3,1,9928-BZVLZ,Female,0,No,No,12,No,...,Yes,No,Yes,No,49.85,552.1,No,DSL,Two year,Mailed check
7041,2,3,1,9993-LHIEB,Male,0,Yes,Yes,67,Yes,...,Yes,No,Yes,No,67.85,4627.65,No,DSL,Two year,Mailed check


In [18]:
telco_df = telco_df[telco_df['contract_type_id']== 3] # select out 2-year contracts

In [19]:
telco_df = telco_df[['customer_id','monthly_charges','tenure','total_charges']] # only use the columns we want

In [20]:
telco_df 

,customer_id,monthly_charges,tenure,total_charges
5348,0040-HALCW,20.40,54,1090.6
5349,0042-RLHYP,19.70,69,1396.9
5350,0057-QBUQH,25.10,43,1070.15
5351,0064-SUDOG,20.30,12,224.5
5352,0071-NDAFP,25.50,25,630.6
...,...,...,...,...
7038,9919-FZDED,84.10,72,5981.65
7039,9926-PJHDQ,76.80,72,5468.45
7040,9928-BZVLZ,49.85,12,552.1
7041,9993-LHIEB,67.85,67,4627.65


### 2. Using your acquired Telco data, walk through the summarization and cleaning steps in your wrangle.ipynb file like we did above. You may handle the missing values however you feel is appropriate and meaningful; remember to document your process and decisions using markdown and code commenting where helpful.

In [21]:
telco_df.info()# check info for dtypes and empty fields. Lack of a null means that the field is a space

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1695 entries, 5348 to 7042
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1695 non-null   object 
 1   monthly_charges  1695 non-null   float64
 2   tenure           1695 non-null   int64  
 3   total_charges    1695 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 66.2+ KB


In [22]:
telco_df.describe().T # right now, it will only show monthly_charges and tenure because theyre they only num types

,count,mean,std,min,25%,50%,75%,max
monthly_charges,1695.0,60.770413,34.678865,18.4,24.025,64.35,90.45,118.75
tenure,1695.0,56.735103,18.209363,0.0,48.000,64.00,71.00,72.00


In [23]:
telco_df['total_charges'] = telco_df.total_charges.str.replace(' ','0.00').astype('float64') # replace empty fields in total charges with 0, and convert to a float

In [24]:
telco_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1695 entries, 5348 to 7042
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1695 non-null   object 
 1   monthly_charges  1695 non-null   float64
 2   tenure           1695 non-null   int64  
 3   total_charges    1695 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 66.2+ KB


In [25]:
telco_df.tenure.astype('float64') # change tenure to a float so it matches the other nums

5348    54.0
5349    69.0
5350    43.0
5351    12.0
5352    25.0
        ... 
7038    72.0
7039    72.0
7040    12.0
7041    67.0
7042    63.0
Name: tenure, Length: 1695, dtype: float64

In [12]:
# create a function to just clean the data
def telco_clean(telco_df):
    telco_df = telco_df[telco_df['contract_type_id']== 3]
    telco_df = telco_df[['customer_id','monthly_charges','tenure','total_charges']]
    telco_df['total_charges'] = telco_df.total_charges.str.replace(' ','0.00').astype('float64')
    telco_df.tenure = telco_df.tenure.astype('float64')
    return telco_df

In [13]:
telco_clean(telco_df)# test it

AttributeError: Can only use .str accessor with string values!

### 3. End with a wrangle.py file that contains the necessary functions to automate your process from acquiring the data to returning a cleaned dataframe with no missing values. Name your final function wrangle_telco.

In [27]:
wrangle.wrangle_telco() # combine it with the get_telco_db function to put it all in one step

,customer_id,monthly_charges,tenure,total_charges
5348,0040-HALCW,20.40,54.0,1090.60
5349,0042-RLHYP,19.70,69.0,1396.90
5350,0057-QBUQH,25.10,43.0,1070.15
5351,0064-SUDOG,20.30,12.0,224.50
5352,0071-NDAFP,25.50,25.0,630.60
...,...,...,...,...
7038,9919-FZDED,84.10,72.0,5981.65
7039,9926-PJHDQ,76.80,72.0,5468.45
7040,9928-BZVLZ,49.85,12.0,552.10
7041,9993-LHIEB,67.85,67.0,4627.65


# Exercises III

> Let's set up an example scenario as perspective for our regression exercises using the Zillow dataset.
As a Codeup data science graduate, you want to show off your skills to the Zillow data science team in hopes of getting an interview for a position you saw pop up on LinkedIn. You thought it might look impressive to build an end-to-end project in which you use some of their Kaggle data to predict property values using some of their available features; who knows, you might even do some feature engineering to blow them away. Your goal is to predict the values of single unit properties using the obervations from 2017.
In these exercises, you will complete the first step toward the above goal: acquire and prepare the necessary Zillow data from the zillow database in the Codeup database server.

### 1. Acquire bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, and fips from the zillow database for all 'Single Family Residential' properties.

In [3]:
zillow_df = acquire.get_zillow_db()

KeyboardInterrupt: 

In [ ]:
zillow_df.head(3)

2. Using your acquired Zillow data, walk through the summarization and cleaning steps in your wrangle.ipynb file like we did above. You may handle the missing values however you feel is appropriate and meaninful; remember to document your process and decisions using markdown and code commenting where helpful.

In [ ]:
zillow_df.describe().T

In [ ]:
zillow_df.info(null_counts=True)

### Initial thoughts
- plenty of null values
- min values for bedroom and bathroom counts are 0, either they are nulls or there is some wierd entries
- plenty of rows are not complete, the counts for each value vary wildly 
- dtypes are all floats, so we wont have to convert anything. 

### Next steps?
- drop nulls
- drop dupes

In [ ]:
def missing_values_table(df):
    '''
    this function takes a dataframe as input and will output metrics for missing values, and the percent of that column that has missing values
    '''
        # Total missing values
    mis_val = df.isnull().sum()
        # Percentage of missing values
    mis_val_percent = 100 * df.isnull().sum() / len(df)
        # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        # Sort the table by percentage of missing descending
    mis_val_table_ren_columns = mis_val_table_ren_columns[
    mis_val_table_ren_columns.iloc[:,1] != 0].sort_values('% of Total Values', ascending=False).round(1)
        # Print some summary information
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
           "There are " + str(mis_val_table_ren_columns.shape[0]) +
           "columns that have missing values.")
        # Return the dataframe with missing information
    return mis_val_table_ren_columns

In [ ]:
missing_values_table(zillow_df)

In [ ]:
zillow_df.isnull().sum() # count out the nulls. If we drop all the nulls, we'd still have over 2 million entries

In [ ]:
zillow_df = zillow_df.dropna()

In [ ]:
zillow_df.isnull().sum()

In [ ]:
zillow_df.info(null_counts=True)

> Even after dropping, we still have 2.1 million rows to work with

In [ ]:
zillow_df.to_csv('zillow.csv') # cache it to speed up import

In [ ]:
zillow_df.describe().T

In [ ]:
zillow_df.head()

In [ ]:
zillow_df.groupby(zillow_df.columns.tolist(),as_index=False).size() # check for duplicates

In [ ]:
zillow_df = zillow_df.drop_duplicates() # drop the duplicates

> after dropping duplicates, we still have 2.1 million rows. Plenty to work with

In [ ]:
zillow_df.to_csv('zillow.csv')# save to csv again. remember to delete the original one

In [ ]:
wrangle.wrangle_zillow() # place all the cleaning steps and import steps into a function, and test.

> it works!